In [34]:
# https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/recurrent_network.py

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import math

mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [35]:
learning_rate = 0.1
epochs = 18
batch_size = 128

num_input = 28
timesteps = 28
num_hidden = 128
num_classes = 10

x = tf.placeholder(tf.float32, [None, num_input, timesteps])
y = tf.placeholder(tf.float32, [None, num_classes])

with tf.variable_scope('rnn_weights_biases'):
    weight_out = tf.Variable(tf.truncated_normal([num_hidden, num_classes]))
    bias_out = tf.Variable(tf.truncated_normal([num_classes]))

x_ = tf.unstack(x, timesteps, 1)

lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_hidden, forget_bias=1.0, reuse=True)
outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x_, dtype=tf.float32)

logits = tf.add(tf.matmul(outputs[-1], weight_out), bias_out)
y_ = tf.nn.softmax(logits)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for e in range(epochs):
        batches = int(math.ceil(mnist.train.images.shape[0] / float(batch_size)))
        for batch_i in range(batches):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            batch_x = batch_x.reshape([batch_size, timesteps, num_input])
            _, loss = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})

            if batch_i % 100 == 0:
                loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y})
                print("Epoch: {}/{}, loss: {:.5f}, acc: {:.5f}".format(e + 1, epochs, loss, acc))
    
    
    saver.save(sess, 'checkpoints/mnist.rnn.tf.ckpt')


Epoch: 1/18, loss: 2.32749, acc: 0.08594
Epoch: 1/18, loss: 2.17113, acc: 0.30469
Epoch: 1/18, loss: 1.99007, acc: 0.47656
Epoch: 1/18, loss: 1.95847, acc: 0.52344
Epoch: 1/18, loss: 1.92955, acc: 0.54688
Epoch: 2/18, loss: 1.82744, acc: 0.65625
Epoch: 2/18, loss: 1.83711, acc: 0.64062
Epoch: 2/18, loss: 1.74376, acc: 0.71875
Epoch: 2/18, loss: 1.68114, acc: 0.78906
Epoch: 2/18, loss: 1.64556, acc: 0.82812
Epoch: 3/18, loss: 1.64400, acc: 0.82031
Epoch: 3/18, loss: 1.58300, acc: 0.89062
Epoch: 3/18, loss: 1.59178, acc: 0.87500
Epoch: 3/18, loss: 1.60885, acc: 0.87500
Epoch: 3/18, loss: 1.57090, acc: 0.89844
Epoch: 4/18, loss: 1.60909, acc: 0.85938
Epoch: 4/18, loss: 1.57759, acc: 0.89062
Epoch: 4/18, loss: 1.58779, acc: 0.87500
Epoch: 4/18, loss: 1.59188, acc: 0.85938
Epoch: 4/18, loss: 1.61787, acc: 0.84375
Epoch: 5/18, loss: 1.54573, acc: 0.92969
Epoch: 5/18, loss: 1.54813, acc: 0.92188
Epoch: 5/18, loss: 1.58776, acc: 0.88281
Epoch: 5/18, loss: 1.61732, acc: 0.84375
Epoch: 5/18, los

In [36]:
with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints/'))

    x_test = mnist.test.images.reshape((-1, timesteps, num_input))
    y_test = mnist.test.labels
    test_acc = sess.run(accuracy, feed_dict={x: x_test, y: y_test})
    print("Test Accuracy: {:.5f}".format(test_acc))

INFO:tensorflow:Restoring parameters from checkpoints/mnist.rnn.tf.ckpt
Test Accuracy: 0.97370
